# Lesson 1: Router Engine

Welcome to Lesson 1.

To access the `requirements.txt` file, the data/pdf file required for this lesson and the `helper` and `utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [1]:
from dotenv import load_dotenv
import os

load_dotenv("../../.env")
token = os.getenv("HUGGINGFACEHUB_API_TOKEN")


True

In [2]:
from llama_index.core import Settings
from llama_index.llms.huggingface import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = HuggingFaceInferenceAPI(
    model_name="mistralai/Mistral-7B-Instruct-v0.3",
    token=token,
)
Settings.embed_model = HuggingFaceEmbedding(
    model_name = 'sentence-transformers/all-MiniLM-L6-v2',
)

In [3]:
import nest_asyncio

nest_asyncio.apply()

## Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

In [4]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

## Define LLM and Embedding model

In [5]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

## Define Summary Index and Vector Index over the Same Data

In [7]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [8]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [9]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Define Router Query Engine

In [10]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [11]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Choice 1 is useful for summarizing the document as a whole, while choice 2 is more specific to retrieving specific context within the document..
 The document introduces MetaGPT, a framework for software development that utilizes a specialized division of labor and SOPs workflow to design software through natural language programming. MetaGPT's structured messaging and feedback mechanisms reduce the loss of communication information and improve the execution of code. The framework was tested on the SoftwareDev dataset, and its performance was compared to Chat-Dev. MetaGPT was also tested with different LLMs, and it was found that using GPT-4 as the backend yields superior performance. The impact of instruction levels (high-level vs detailed instructions) was also investigated, and it was found that detailed prompts lead to better software projects with lower productivity ratios. MetaGPT addresses challenges such as use context efficiently, reduce hallucination

In [12]:
print(len(response.source_nodes))

34


In [13]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 0: The question is related to the interaction between agents, which is a common topic in the context of MetaGPT. However, the question is not specific to summarization or retrieval of information from the paper, but rather about understanding the content of the paper. Therefore, choice 1 is the most relevant..
1. Based on the context, it is not explicitly mentioned how agents share information with other agents. However, it is possible that they might share information through network connections or APIs, as the context mentions URL errors and dependencies.
2. Another possibility is that they might share information through files, as the context mentions PNG files missing and compile bug fixes.
3. Lastly, it is also possible that they might share information through a common main function, as the context mentions a missing main function error.
4. It is also worth noting that the context mentions dependency errors, which could indicate that the agents might be usi

## Let's put everything together

In [14]:
from utils import get_router_query_engine_hf

query_engine = get_router_query_engine_hf("metagpt.pdf")

d:\gits\GenAI-w-langchain-and-llamaindex\Llamaindex\Agentic RAG\lesson1\utils.py:70: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPI. (Deprecated in favor of `HuggingFaceInferenceAPI` from `llama-index-llms-huggingface-api` which should be used instead.)
  Settings.llm = HuggingFaceInferenceAPI(
c:\Users\Ghost\anaconda3\envs\rag\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
response = query_engine.query("Give brief description about the paper.")
print(str(response))

Selecting query engine 1: Choice 2 is more specific to the question, as it refers to the paper itself..
 The paper presents MetaGPT, a large language model that can generate code, design software architecture, and develop standard operating procedures (SOPs) for various tasks. The model is designed to self-improve and learn from feedback, and it can also generate test cases for the code it generates. The paper describes various experiments conducted to evaluate the performance of MetaGPT, including comparisons with other large language models and open-source baseline methods. The authors also discuss the potential applications of MetaGPT in software development, including the automation of software development tasks and the generation of SOPs for effective communication in teams.
